In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery

# Snce we don't have a dedicated function in the package to read the FX table (we only have one to write it),  
# we will use the BigQuery Client directly. It proves the Cloud Data Warehouse is actually working.

# 1. Setup - Force ID just to be safe in the notebook environment
PROJECT_ID = "projectbdm-487109"
DATASET = "patek_data"
TABLE = "fx_rates"
os.environ["GCP_PROJECT_ID"] = PROJECT_ID

print("⏳ Connecting to BigQuery to fetch FX rates...")
client = bigquery.Client()

# 2. Query
# We order by date to make sure the line chart connects points correctly
query = f"""
    SELECT date, base_currency, rate
    FROM `{PROJECT_ID}.{DATASET}.{TABLE}`
    WHERE target_currency = 'EUR'
    ORDER BY date ASC
"""

df_fx = client.query(query).to_dataframe()

# 3. Clean Types (BigQuery sometimes sends dates as Objects)
df_fx['date'] = pd.to_datetime(df_fx['date'])
df_fx['rate'] = pd.to_numeric(df_fx['rate'])

print(f"✅ Loaded {len(df_fx)} daily rates.")
print(df_fx.head())

/usr/local/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


ValueError: Missing env var GCP_PROJECT_ID

In [ ]:
# 1. Filter for major currencies to keep the chart readable
major_currencies = ['USD', 'GBP', 'CHF', 'JPY', 'CNY']
df_subset = df_fx[df_fx['base_currency'].isin(major_currencies)].copy()

# 2. Plotting
plt.figure(figsize=(14, 7))

# We use 'hue' to automatically color lines by currency
sns.lineplot(data=df_subset, x='date', y='rate', hue='base_currency', linewidth=2.5)

# 3. Styling
plt.title("Evolution of Major Currencies vs EUR (2020-2023)", fontsize=16)
plt.ylabel("Exchange Rate (1 Unit = X EUR)", fontsize=12)
plt.xlabel("Date", fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(title="Currency", loc='upper left')

# Optional: Add a reference line for Parity (1.0)
plt.axhline(1.0, color='black', linestyle='--', alpha=0.5, label='Parity')

plt.show()